In [1]:
import pandas as pd
import csv
import io
from typing import Dict, List

class DetailedExcelValidator:
    """첫 번째 코드에서 생성된 detailed Excel을 직접 활용하는 검증기 - 100% 정확함"""
    
    def __init__(self, exclude_keys=None):
        self.test_cases = []
        self.exclude_keys = exclude_keys if exclude_keys is not None else ['page_url', 'os_name']
        
    def validate_using_detailed_excel(self, detailed_excel_path: str, tsv_path: str, output_file: str):
        """detailed Excel과 TSV를 함께 사용하여 검증 - 근본적 해결책"""
        try:
            # 1. Detailed Excel 로드
            detailed_df = pd.read_excel(detailed_excel_path, engine='openpyxl')
            print(f"✅ Detailed Excel 로드 완료: {len(detailed_df)}개 key-value 쌍")
            
            # 2. TSV 파일 로드
            tsv_df = pd.read_csv(tsv_path, sep='\t', encoding='utf-8', header=0)
            if len(tsv_df.columns) == 19:
                tsv_df = tsv_df.iloc[:,:-4]
            
            tsv_df.columns = tsv_df.columns.str.rstrip(',').str.strip()
            
            for col in tsv_df.columns:
                if tsv_df[col].dtype == 'object':
                    tsv_df[col] = tsv_df[col].astype(str).str.rstrip(',').str.strip()
            
            print(f"✅ TSV 로드 완료: {len(tsv_df)}개 테스트 케이스")
            
            # 3. 로그 번호별로 실제 키-값 매핑 생성
            log_key_value_map = {}
            for idx, row in detailed_df.iterrows():
                log_no = row['no']
                key = row['key']
                value = str(row['value']) if pd.notna(row['value']) else ''
                
                if log_no not in log_key_value_map:
                    log_key_value_map[log_no] = {}
                
                log_key_value_map[log_no][key] = value
            
            print(f"✅ 키-값 매핑 생성 완료: {len(log_key_value_map)}개 로그")
            
            # 4. 검증 실행
            results = []
            
            def safe_str(value):
                if pd.isna(value) or str(value).lower() == 'nan':
                    return ''
                return str(value).strip()
            
            def compare_values(actual, expected):
                """두 값을 비교하여 PASS/FAIL 반환"""
                actual_clean = safe_str(actual)
                expected_clean = safe_str(expected)
                
                # 둘 다 비어있으면 PASS
                if not actual_clean and not expected_clean:
                    return 'PASS'
                # 하나만 비어있거나 값이 다르면 FAIL
                elif actual_clean != expected_clean:
                    return 'FAIL'
                else:
                    return 'PASS'
            
            # TSV의 각 행과 매칭하여 검증
            for idx, row in tsv_df.iterrows():
                unique_id = idx + 1
                
                # TSV에서 정답 키들 추출
                expected_keys_str = safe_str(row['key'])
                expected_keys = [k.strip() for k in expected_keys_str.split(',') if k.strip() and k.strip().lower() != 'nan']
                
                # TSV에서 정답 값들 추출
                expected_values_str = safe_str(row['value'])
                expected_values = self._parse_expected_values(expected_values_str, len(expected_keys))
                
                # detailed Excel에서 해당 로그의 실제 키-값 가져오기
                actual_key_value_map = log_key_value_map.get(unique_id, {})
                
                # 제외 키 필터링
                filtered_actual_keys = [k for k in actual_key_value_map.keys() if k not in self.exclude_keys]
                
                # 디버깅용 출력 (11번 행만)
                if unique_id == 11:
                    print(f"\n=== 디버깅 정보 (로그 {unique_id}) ===")
                    print(f"expected_keys: {expected_keys}")
                    print(f"expected_values: {expected_values}")
                    print(f"actual_key_value_map 샘플:")
                    for i, (k, v) in enumerate(list(actual_key_value_map.items())[:5]):
                        print(f"  [{i}] {k}: '{v}'")
                    print(f"실제 키 개수: {len(actual_key_value_map)}")
                    print(f"필터링된 키 개수: {len(filtered_actual_keys)}")
                
                # 정답 키-값 매핑 생성
                expected_key_value_map = {}
                for i, key in enumerate(expected_keys):
                    if i < len(expected_values):
                        expected_key_value_map[key] = safe_str(expected_values[i])
                    else:
                        expected_key_value_map[key] = ''
                
                # 로그가 없는 경우 처리
                if len(actual_key_value_map) == 0 and len(expected_keys) > 0:
                    for key in expected_keys:
                        expected_value = expected_key_value_map.get(key, '')
                        results.append({
                            '고유번호': unique_id,
                            '기능': safe_str(row['기능']),
                            '경로': safe_str(row['경로']),
                            '활동': safe_str(row['활동']),
                            'page_id': safe_str(row['page_id']),
                            'act_type': safe_str(row['act_type.1']) if 'act_type.1' in row else safe_str(row.get('act_type', '')),
                            'act_type_정답': safe_str(row.get('act_type', '')),
                            'click_type': safe_str(row['click_type.1']) if 'click_type.1' in row else safe_str(row.get('click_type', '')),
                            'click_type_정답': safe_str(row.get('click_type', '')),
                            'click_text': safe_str(row.get('click_text', '')),
                            'key': '',
                            'key_정답': key,
                            'value': '',
                            'value_정답': expected_value,
                            'key-pass': 'FAIL',
                            'value-pass': 'FAIL',
                            'exist-pass': 'MISSING'
                        })
                    continue
                
                if len(actual_key_value_map) == 0 and len(expected_keys) == 0:
                    results.append({
                        '고유번호': unique_id,
                        '기능': safe_str(row['기능']),
                        '경로': safe_str(row['경로']),
                        '활동': safe_str(row['활동']),
                        'page_id': safe_str(row['page_id']),
                        'act_type': safe_str(row['act_type.1']) if 'act_type.1' in row else safe_str(row.get('act_type', '')),
                        'act_type_정답': safe_str(row.get('act_type', '')),
                        'click_type': safe_str(row['click_type.1']) if 'click_type.1' in row else safe_str(row.get('click_type', '')),
                        'click_type_정답': safe_str(row.get('click_type', '')),
                        'click_text': safe_str(row.get('click_text', '')),
                        'key': '로그 누락',
                        'key_정답': 'NO_LOG_SPEC',
                        'value': '로그 사양 없음',
                        'value_정답': safe_str(row['value']),
                        'key-pass': 'REVIEW',
                        'value-pass': 'REVIEW',
                        'exist-pass': 'REVIEW'
                    })
                    continue
                
                # 정상 검증 케이스
                actual_keys_set = set(actual_key_value_map.keys())
                expected_keys_set = set(expected_keys)
                
                # 정답 키들에 대한 검증
                for key in expected_keys:
                    actual_value = actual_key_value_map.get(key, '')
                    expected_value = expected_key_value_map.get(key, '')
                    
                    key_exists = key in actual_keys_set
                    actual_key = key if key_exists else ''
                    
                    # 특별한 키들에 대한 정답값 처리
                    if key == 'page_id':
                        expected_value = safe_str(row['page_id'])
                    elif key == 'act_type':
                        expected_value = safe_str(row.get('act_type', ''))
                    elif key == 'click_type':
                        expected_value = safe_str(row.get('click_type', ''))
                    
                    # 키와 값 평가
                    key_pass = 'PASS' if key_exists else 'FAIL'
                    exist_pass = 'PASS' if key_exists else 'MISSING'
                    
                    if not key_exists:
                        value_pass = 'FAIL'
                    else:
                        value_pass = compare_values(actual_value, expected_value)
                    
                    results.append({
                        '고유번호': unique_id,
                        '기능': safe_str(row['기능']),
                        '경로': safe_str(row['경로']),
                        '활동': safe_str(row['활동']),
                        'page_id': safe_str(row['page_id']),
                        'act_type': safe_str(row['act_type.1']) if 'act_type.1' in row else safe_str(row.get('act_type', '')),
                        'act_type_정답': safe_str(row.get('act_type', '')),
                        'click_type': safe_str(row['click_type.1']) if 'click_type.1' in row else safe_str(row.get('click_type', '')),
                        'click_type_정답': safe_str(row.get('click_type', '')),
                        'click_text': safe_str(row.get('click_text', '')),
                        'key': actual_key,
                        'key_정답': key,
                        'value': safe_str(actual_value),
                        'value_정답': expected_value,
                        'key-pass': key_pass,
                        'value-pass': value_pass,
                        'exist-pass': exist_pass
                    })
                
                # UNEXPECTED 키 처리 (제외 키 제외)
                unexpected_keys = set(filtered_actual_keys) - expected_keys_set
                for key in unexpected_keys:
                    actual_value = actual_key_value_map.get(key, '')
                    
                    results.append({
                        '고유번호': unique_id,
                        '기능': safe_str(row['기능']),
                        '경로': safe_str(row['경로']),
                        '활동': safe_str(row['활동']),
                        'page_id': safe_str(row['page_id']),
                        'act_type': safe_str(row['act_type.1']) if 'act_type.1' in row else safe_str(row.get('act_type', '')),
                        'act_type_정답': safe_str(row.get('act_type', '')),
                        'click_type': safe_str(row['click_type.1']) if 'click_type.1' in row else safe_str(row.get('click_type', '')),
                        'click_type_정답': safe_str(row.get('click_type', '')),
                        'click_text': safe_str(row.get('click_text', '')),
                        'key': key,
                        'key_정답': '',
                        'value': safe_str(actual_value),
                        'value_정답': '',
                        'key-pass': 'UNEXPECTED',
                        'value-pass': 'UNEXPECTED',
                        'exist-pass': 'UNEXPECTED'
                    })
            
            # 5. 결과 저장
            df_results = pd.DataFrame(results)
            
            # 컬럼 순서 정의
            column_order = [
                '고유번호', '기능', '경로', '활동', 'page_id', 
                'click_text',
                'act_type_정답',
                'act_type',
                'click_type_정답',
                'click_type',
                'key_정답',
                'key',
                'value_정답',
                'value',
                'key-pass',
                'value-pass',
                'exist-pass'
            ]
            
            df_results = df_results[column_order]
            df_results_retry = df_results.iloc[:,:14]
            df_results.to_excel(output_file, index=False, engine='openpyxl')
            df_results_retry.to_excel("./result/qa_result_retry_detailed.xlsx", index=False, engine='openpyxl')
            
            # 6. 통계 출력
            total = len(df_results)
            key_passed = len(df_results[df_results['key-pass'] == 'PASS'])
            value_passed = len(df_results[df_results['value-pass'] == 'PASS'])
            print(f"\n✅ 검증 완료 (Detailed Excel 기반):")
            print(f"   전체: {total}개")
            print(f"   키 통과: {key_passed}개 ({key_passed/total*100:.1f}%)")
            print(f"   값 통과: {value_passed}개 ({value_passed/total*100:.1f}%)")
            
            return results
            
        except Exception as e:
            print(f"❌ 오류: {e}")
            import traceback
            traceback.print_exc()
            raise
    
    def _parse_expected_values(self, values_str: str, expected_count: int):
        """정답 값들을 파싱 (템플릿 문자 고려)"""
        if not values_str or expected_count <= 0:
            return []
        
        try:
            # 쉼표로 분할
            split_values = [v.strip() for v in values_str.split(',')]
            
            # 개수 맞추기
            if len(split_values) == expected_count:
                return split_values
            elif len(split_values) > expected_count:
                # 초과된 값들을 마지막 항목에 병합
                result = split_values[:expected_count-1]
                remaining = ', '.join(split_values[expected_count-1:])
                result.append(remaining)
                return result
            else:
                # 부족한 개수만큼 빈 문자열 추가
                while len(split_values) < expected_count:
                    split_values.append('')
                return split_values
                
        except Exception:
            # 파싱 실패 시 빈 값들로 채움
            return [''] * expected_count

# 사용 예시
def run_detailed_validator():
    """Detailed Excel 기반 검증기 실행 - 근본적 해결책"""
    validator = DetailedExcelValidator(exclude_keys=['page_url', 'os_name'])
    validator.validate_using_detailed_excel(
        "./result/log_analysis_detailed.xlsx",  # 첫 번째 코드에서 생성된 detailed Excel
        "./notion-log_tsv/tester.tsv",          # 테스트 케이스 TSV
        "./result/qa_result_detailed_based.xlsx"  # 검증 결과 출력
    )

if __name__ == "__main__":
    run_detailed_validator()

✅ Detailed Excel 로드 완료: 959개 key-value 쌍
✅ TSV 로드 완료: 155개 테스트 케이스
✅ 키-값 매핑 생성 완료: 87개 로그

=== 디버깅 정보 (로그 11) ===
expected_keys: ['page_id', 'area_name', 'area_order', 'prd_code', 'prd_order', 'prd_brand', 'prd_name', 'prd_price_origin', 'prd_price_final', 'prd_disc_rate', 'prd_tag', 'prd_review_score', 'prd_review_cnt', 'prd_is_ad']
expected_values: ['{{page_id}}', '{{영역 TEXT}}', '{{영역 순서}}', '{{상품 코드}}', '{{상품 순서}}', '{{브랜드}}', '{{상품명}}', '{{가격}}', '{{할인 가격}}', '{{할인율}}', '{{상품 태그}}', '{{별 점}}', '{{리뷰 수}}', '{{광고 상품 여부}}']
actual_key_value_map 샘플:
  [0] channel: 'Rround'
  [1] page_url: 'https://store.rround.com/main/home'
  [2] page_id: 'store/main/home'
  [3] act_type: 'click'
  [4] click_text: '바비리스 버터 바 스트레이트너 ST520K'
실제 키 개수: 19
필터링된 키 개수: 17

✅ 검증 완료 (Detailed Excel 기반):
   전체: 1323개
   키 통과: 428개 (32.4%)
   값 통과: 87개 (6.6%)
